In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import re
import random

from IPython.display import display

from nltk import word_tokenize, sent_tokenize

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV,StratifiedKFold
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support, classification_report
from sklearn.dummy import DummyClassifier
import nltk

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train_url = 'https://media.githubusercontent.com/media/ricadolcvuba/Dataset-TP2/main/df_train_preprocesado.csv'
df_train = pd.read_csv(train_url)
x_train = df_train['review_es']
y_train = df_train['sentimiento']
test_url = 'https://media.githubusercontent.com/media/ricadolcvuba/Dataset-TP2/main/df_test_preprocesado.csv'
df_test = pd.read_csv(test_url)
x_test = df_test['review_es']

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
my_stop_words = stopwords.words('spanish')
# Instanciamos el CV
vec = CountVectorizer(stop_words=my_stop_words, ngram_range=(1,2))

# Ajustamos el CV y transformamos los datos
x_train = vec.fit_transform(x_train)
x_test = vec.transform(x_test)

## RN

In [ ]:
!pip install scikeras

In [ ]:
import tensorflow as tf
from tensorflow import keras
from scikeras.wrappers import KerasClassifier
from sklearn.metrics import make_scorer, f1_score
import numpy as np

In [ ]:
# Create the sklearn model for the network

def create_model():
    cant_clases=len(np.unique(y_train))

    modelo= keras.Sequential([
    #keras.layers.Dense(2, activation='sigmoid'),
    keras.Input(shape=(3192097,)),
    keras.layers.Dense(cant_clases, activation='sigmoid')])

    modelo.compile(
      optimizer="adam",
      loss="sparse_categorical_crossentropy",
      # metricas para ir calculando en cada iteracion o batch
      metrics=['f1_score'],
    )

    return modelo


In [ ]:
modelo_cv = KerasClassifier(model=create_model)

In [ ]:
from sklearn.model_selection import StratifiedKFold, KFold,RandomizedSearchCV

epochs = [10, 5]
batches = [35, 50]

param_grid = dict( epochs=epochs, batch_size=batches)

#Cantidad de combinaciones que quiero porbar
n=2
#Cantidad de splits para el Cross Validation
folds=4
#Kfold estratificado
kfoldcv = KFold(n_splits=folds)

#Metrica que quiero optimizar F1 Score
scorer_fn = make_scorer(f1_score, average='micro',pos_label='positivo')

#Random Search Cross Validation
randomcv = RandomizedSearchCV(estimator=modelo_cv,
                              param_distributions = param_grid,
                              scoring=scorer_fn,
                              cv=kfoldcv,
                              random_state=20,
                              n_iter=n,
                              error_score='raise')
#Busco los hiperparamtros que optimizan F1 Score
randomcv.fit(x_train, y_train);

Epoch 1/10
1033/1033 ━━━━━━━━━━━━━━━━━━━━ 73s 70ms/step - f1_score: 0.4982 - loss: 0.4426
Epoch 2/10
1033/1033 ━━━━━━━━━━━━━━━━━━━━ 75s 73ms/step - f1_score: 0.5000 - loss: 0.0430
Epoch 3/10
1033/1033 ━━━━━━━━━━━━━━━━━━━━ 73s 71ms/step - f1_score: 0.4999 - loss: 0.0167
Epoch 4/10
1033/1033 ━━━━━━━━━━━━━━━━━━━━ 85s 82ms/step - f1_score: 0.5000 - loss: 0.0088
Epoch 5/10
1033/1033 ━━━━━━━━━━━━━━━━━━━━ 141s 81ms/step - f1_score: 0.5000 - loss: 0.0052
Epoch 6/10
1033/1033 ━━━━━━━━━━━━━━━━━━━━ 73s 71ms/step - f1_score: 0.5000 - loss: 0.0031
Epoch 7/10
1033/1033 ━━━━━━━━━━━━━━━━━━━━ 77s 75ms/step - f1_score: 0.5000 - loss: 0.0018
Epoch 8/10
1033/1033 ━━━━━━━━━━━━━━━━━━━━ 79s 72ms/step - f1_score: 0.5000 - loss: 0.0012
Epoch 9/10
1033/1033 ━━━━━━━━━━━━━━━━━━━━ 73s 70ms/step - f1_score: 0.5000 - loss: 7.1069e-04
Epoch 10/10
1033/1033 ━━━━━━━━━━━━━━━━━━━━ 82s 70ms/step - f1_score: 0.5000 - loss: 4.4853e-04
345/345 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1569: UserWarning: Note that pos_label (set to 'positivo') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(


Epoch 1/10
1033/1033 ━━━━━━━━━━━━━━━━━━━━ 75s 71ms/step - f1_score: 0.4988 - loss: 0.4407
Epoch 2/10
1033/1033 ━━━━━━━━━━━━━━━━━━━━ 80s 70ms/step - f1_score: 0.4999 - loss: 0.0429
Epoch 3/10
1033/1033 ━━━━━━━━━━━━━━━━━━━━ 82s 70ms/step - f1_score: 0.4999 - loss: 0.0165
Epoch 4/10
1033/1033 ━━━━━━━━━━━━━━━━━━━━ 73s 70ms/step - f1_score: 0.5000 - loss: 0.0088
Epoch 5/10
1033/1033 ━━━━━━━━━━━━━━━━━━━━ 81s 70ms/step - f1_score: 0.5000 - loss: 0.0052
Epoch 6/10
1033/1033 ━━━━━━━━━━━━━━━━━━━━ 82s 70ms/step - f1_score: 0.5000 - loss: 0.0031
Epoch 7/10
1033/1033 ━━━━━━━━━━━━━━━━━━━━ 72s 69ms/step - f1_score: 0.5000 - loss: 0.0019
Epoch 8/10
1033/1033 ━━━━━━━━━━━━━━━━━━━━ 72s 69ms/step - f1_score: 0.5000 - loss: 0.0011
Epoch 9/10
1033/1033 ━━━━━━━━━━━━━━━━━━━━ 72s 69ms/step - f1_score: 0.5000 - loss: 7.2173e-04
Epoch 10/10
1033/1033 ━━━━━━━━━━━━━━━━━━━━ 72s 69ms/step - f1_score: 0.5000 - loss: 4.2541e-04
345/345 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1569: UserWarning: Note that pos_label (set to 'positivo') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(


Epoch 1/10
1033/1033 ━━━━━━━━━━━━━━━━━━━━ 81s 77ms/step - f1_score: 0.4983 - loss: 0.4402
Epoch 2/10
1033/1033 ━━━━━━━━━━━━━━━━━━━━ 73s 69ms/step - f1_score: 0.5000 - loss: 0.0437
Epoch 3/10
1033/1033 ━━━━━━━━━━━━━━━━━━━━ 82s 69ms/step - f1_score: 0.4999 - loss: 0.0163
Epoch 4/10
1033/1033 ━━━━━━━━━━━━━━━━━━━━ 72s 69ms/step - f1_score: 0.5000 - loss: 0.0088
Epoch 5/10
1033/1033 ━━━━━━━━━━━━━━━━━━━━ 82s 69ms/step - f1_score: 0.5000 - loss: 0.0051
Epoch 6/10
1033/1033 ━━━━━━━━━━━━━━━━━━━━ 82s 69ms/step - f1_score: 0.5000 - loss: 0.0030
Epoch 7/10
1033/1033 ━━━━━━━━━━━━━━━━━━━━ 82s 70ms/step - f1_score: 0.5000 - loss: 0.0019
Epoch 8/10
1033/1033 ━━━━━━━━━━━━━━━━━━━━ 82s 70ms/step - f1_score: 0.5000 - loss: 0.0011
Epoch 9/10
1033/1033 ━━━━━━━━━━━━━━━━━━━━ 82s 69ms/step - f1_score: 0.5000 - loss: 7.0127e-04
Epoch 10/10
1033/1033 ━━━━━━━━━━━━━━━━━━━━ 72s 70ms/step - f1_score: 0.5000 - loss: 4.3207e-04
345/345 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step


...

Epoch 1/5
1033/1033 ━━━━━━━━━━━━━━━━━━━━ 73s 69ms/step - f1_score: 0.4950 - loss: 0.4425
Epoch 2/5
1033/1033 ━━━━━━━━━━━━━━━━━━━━ 72s 69ms/step - f1_score: 0.5001 - loss: 0.0434
Epoch 3/5
1033/1033 ━━━━━━━━━━━━━━━━━━━━ 82s 70ms/step - f1_score: 0.5000 - loss: 0.0164
Epoch 4/5
1033/1033 ━━━━━━━━━━━━━━━━━━━━ 72s 70ms/step - f1_score: 0.5000 - loss: 0.0087
Epoch 5/5
1033/1033 ━━━━━━━━━━━━━━━━━━━━ 82s 70ms/step - f1_score: 0.5000 - loss: 0.0051
345/345 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1569: UserWarning: Note that pos_label (set to 'positivo') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(


Epoch 1/5
1033/1033 ━━━━━━━━━━━━━━━━━━━━ 73s 70ms/step - f1_score: 0.4975 - loss: 0.4421
Epoch 2/5
1033/1033 ━━━━━━━━━━━━━━━━━━━━ 73s 70ms/step - f1_score: 0.4998 - loss: 0.0445
Epoch 3/5
1033/1033 ━━━━━━━━━━━━━━━━━━━━ 72s 70ms/step - f1_score: 0.5000 - loss: 0.0168
Epoch 4/5
1033/1033 ━━━━━━━━━━━━━━━━━━━━ 82s 69ms/step - f1_score: 0.5000 - loss: 0.0087
Epoch 5/5
1033/1033 ━━━━━━━━━━━━━━━━━━━━ 72s 69ms/step - f1_score: 0.5000 - loss: 0.0051
345/345 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1569: UserWarning: Note that pos_label (set to 'positivo') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(


Epoch 1/5
1377/1377 ━━━━━━━━━━━━━━━━━━━━ 104s 74ms/step - f1_score: 0.4974 - loss: 0.4161
Epoch 2/5
1377/1377 ━━━━━━━━━━━━━━━━━━━━ 98s 71ms/step - f1_score: 0.4997 - loss: 0.0369
Epoch 3/5
1377/1377 ━━━━━━━━━━━━━━━━━━━━ 100s 73ms/step - f1_score: 0.5000 - loss: 0.0134
Epoch 4/5
1377/1377 ━━━━━━━━━━━━━━━━━━━━ 145s 75ms/step - f1_score: 0.5000 - loss: 0.0065
Epoch 5/5
1377/1377 ━━━━━━━━━━━━━━━━━━━━ 140s 74ms/step - f1_score: 0.5000 - loss: 0.0034


In [ ]:
#Mejores hiperparametros
print(randomcv.best_params_)
#Mejor métrica
print(randomcv.best_score_)

{'epochs': 5, 'batch_size': 35}
0.8932314614643222


In [ ]:
#Mejor estimador
best_keras=randomcv.best_estimator_

#Predicción con mejor estimador
y_pred_keras=best_keras.predict(x_test)

246/246 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step


In [ ]:
df_pred_keras = pd.DataFrame()
df_pred_keras['ID'] = df_test['ID']
df_pred_keras['sentimiento'] = y_pred_keras

In [ ]:
df_pred_keras.to_csv('df_pred_rn_keras_tf.csv', index=False)

In [ ]:
import joblib

In [ ]:
joblib.dump(best_keras,'rn_keras_tf.joblib')

['rn_keras_tf.joblib']